In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import io
from multiprocessing import Pool

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"

In [3]:
response = requests.get(url)
soup = BeautifulSoup(response.text,"html.parser")

In [7]:
header = soup.find_all('th')
headerList = []
for x in header:
    headerList.append(x.get_text(strip=True))
headerList = headerList[0:9]
d = soup.find('table')
headerList

['Symbol',
 'Security',
 'SEC filings',
 'GICSSector',
 'GICS Sub-Industry',
 'Headquarters Location',
 'Date first added',
 'CIK',
 'Founded']

In [13]:
data = []
for row in d.find_all('tr'):
    sampleList = []
    for td in row.find_all('td'):
        sampleList.append(td.get_text(strip=True))
    data.append(sampleList)
data.pop(0)
df = pd.DataFrame(data, columns = headerList)
df.head()

,Symbol,Security,SEC filings,GICSSector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",1976-08-09,0000066740,1902
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,0000001800,1888
2,ABBV,AbbVie Inc.,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,0001551152,2013 (1888)
3,ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,0000815094,1981
4,ACN,Accenture,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,0001467373,1989


In [14]:
symbol = df['Symbol']
period1 = "1483228800"
period2 = "1609545600"

In [15]:
def scrape(l):
    link = "https://query1.finance.yahoo.com/v7/finance/download/"+l+"?period1="+period1+"&period2="+period2+"&interval=1wk&events=history&includeAdjustedClose=true"
    response = requests.get(link)
    bytes_io = io.BytesIO(response.content)
    dataframe = pd.read_csv(bytes_io, error_bad_lines=False)
    dataframe['Symbol'] = l
    return dataframe

In [ ]:
if __name__ == '__main__':
    p = Pool(30)
    df_list = p.map(scrape,symbol)
    p.close()
    for t in df_list:
        if len(t.columns) > 3:
            close = t['Close'].tolist()
            gl = []
            for i in range(1,len(close)):
                gl.append(((close[i]-close[i-1])/close[i-1])*100)
            gl.insert(0,((gl[0]+gl[1]+gl[2])/3)*100)
            t['Gain or Loss'] = gl

    table = pd.concat(df_list).reset_index(drop=True)

    table.to_csv('Weekly.csv')